### Configuration des ressources

    souscription

In [43]:
%%bash

# CHECK SUBSCRIPTION
az account list \
    --query "[?isDefault].{Name:name, isDefault:isDefault}" \
    --output table

Name                              IsDefault
--------------------------------  -----------
Simplon - HDF - Data Engineer P1  True


    resource group

In [ ]:
%%bash
source ../variables.sh

# GENERATE RG
az group create \
    --name $RG_NAME \
    --location $LOCATION \

In [27]:
# CHECK RG
!source ../variables.sh && az group show --name $RG_NAME --query "{Name:name, ProvisioningState:properties.provisioningState}" --output tsv

RG_DBREAU	Succeeded


    data lake

In [ ]:
%%bash
source ../variables.sh

# GENERATE ADLS
az storage account create \
    --name $ADLS_NAME \
    --resource-group $RG_NAME \
    --location $LOCATION \
    --sku Standard_LRS \
    --kind StorageV2 \
    --hns true  #  🚨 hns > true < important pour créer data lake et pas un storage account 


In [28]:
# CHECK ADLS
!source ../variables.sh && az storage account show --name $ADLS_NAME --resource-group $RG_NAME --query "{Name:name, ProvisioningState:provisioningState}" --output tsv

adlsbreau	Succeeded


    container

In [ ]:
%%bash
source ../variables.sh

# GENERATE CNTNR
az storage container create \
    --name $CNTNR_NAME \
    --account-name $ADLS_NAME \
    --auth-mode login \

In [31]:
# CHECK CNTNR
!source ../variables.sh && az storage container show --name $CNTNR_NAME --account-name $ADLS_NAME --auth-mode login --query "{Name:name, State:properties.lease.state}" --output tsv

data	available


    keyvault

<small>création </small>

In [ ]:
%%bash
source ../variables.sh

# GENERATE KV
az keyvault create \
    --name $KV_NAME \
    --resource-group $RG_NAME \
    --location $LOCATION \
    --sku standard \
    --enable-rbac-authorization false \
    --output none

In [44]:
# CHECK KV
!source ../variables.sh && az keyvault show --name $KV_NAME --resource-group $RG_NAME --query "{Name:name, ProvisioningState:properties.provisioningState}" --output tsv

keyvault-dbreau	Succeeded


    service principal data lake

In [110]:
%%bash
source ../variables.sh

# GENERATE SP1 + GET DETAILS
SP_DETAILS=$(az ad sp create-for-rbac \
    --name "$SP_NAME_1" \
    --query "{appId:appId, password:password, tenant:tenant}" \
    -o json)

# ATTACH VARIABLES
APPID=$(echo $SP_DETAILS | jq -r '.appId')
PASSWORD=$(echo $SP_DETAILS | jq -r '.password')
TENANT=$(echo $SP_DETAILS | jq -r '.tenant')

# STORE DETAILS
{
  echo "SP1_ID=$APPID"
  echo "SP_PASSWORD=$PASSWORD"
  echo "TENANT_ID=$TENANT"
} >> ../.env

In [111]:
#CHECK SP1
!source ../variables.sh && az ad sp show --id $(az ad sp list --display-name "$SP_NAME_1" --query "[0].appId" -o tsv) --query "{Name:displayName, Status:accountEnabled}" --output tsv

sp-datalake-dbreau	True


In [114]:
%%bash
source ../variables.sh && source ../.env

# ASSIGN ROLE TO SP1
az role assignment create \
    --assignee $SP1_ID \
    --role "$SP_ROLE_1" \
    --scope "/subscriptions/$(az account show --query id --output tsv)/resourceGroups/$RG_NAME/providers/Microsoft.Storage/storageAccounts/$ADLS_NAME" \
    --query "{RoleDefinitionName:roleDefinitionName, UpdatedOn:updatedOn}" \
    --output table

RoleDefinitionName             UpdatedOn
-----------------------------  --------------------------------
Storage Blob Data Contributor  2025-01-30T13:12:50.853601+00:00


    service principal keyvault

In [146]:
%%bash
source ../variables.sh

# GENERATE SP1 + GET DETAIL
SP_DETAILS=$(az ad sp create-for-rbac \
   --name "$SP_NAME_2" \
   --query "{appId:appId, password:password, tenant:tenant}" \
   -o json)

# ATTACH VARIABLES
APPID=$(echo $SP_DETAILS | jq -r '.appId')
PASSWORD=$(echo $SP_DETAILS | jq -r '.password')
TENANT=$(echo $SP_DETAILS | jq -r '.tenant')

# STORE DETAIL
{
  echo ""
  echo "SP2_ID=$APPID"
} >> ../.env

# SAVE CREDENTIAL IN KEYVAULT
az keyvault secret set \
    --vault-name $KV_NAME \
    --name $KV_SECRET_NAME  \
    --value "$PASSWORD" \
    --tags "type=password" "resource=$SP_NAME_2" "resource_type=service_principal" \
    --query "{name:name, attributes:attributes, tags:tags}" \
    --output table

Name
---------------------------
sp-keyvault-dbreau-password


In [1]:
%%bash
source ../variables.sh && source ../.env

# CHECK KV SECRETS
az keyvault secret list \
    --vault-name $KV_NAME \
    --query "[].{Name:name, Enabled:attributes.enabled, Tags:tags}" \
    --output table

Name                         Enabled
---------------------------  ---------
sp-keyvault-dbreau-password  True
